# Predicting Causal SNPs

In [1]:
# You can get rid of whatever libraries you dont need

import sys
import os
import random
import numpy as np
from numba import jit, prange
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, average_precision_score
from sklearn.preprocessing import RobustScaler, LabelEncoder, StandardScaler, OrdinalEncoder, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.utils import to_undirected, negative_sampling
import networkx as nx
from scipy.spatial import cKDTree
from scipy.special import expit
from typing import List, Dict
import time
import cProfile
import pstats
import io
import category_encoders as ce
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
import copy
from torch_geometric.transforms import RandomNodeSplit
from collections import Counter



# Print versions of imported libraries
print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {matplotlib.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"Torch version: {torch.__version__}")
print(f"Torch Geometric version: {torch_geometric.__version__}")
print(f"NetworkX version: {nx.__version__}")

if torch.cuda.is_available():
    device = torch.device("cuda")          # Current CUDA device
    print(f"Using {torch.cuda.get_device_name()} ({device})")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
else:
    print("CUDA is not available on this device.")

Python version: 3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)]
NumPy version: 1.24.1
Pandas version: 1.5.3
Matplotlib version: 3.7.1
Scikit-learn version: 1.3.0
Torch version: 2.0.1+cu117
Torch Geometric version: 2.3.1
NetworkX version: 3.0
Using NVIDIA RTX A6000 (cuda)
CUDA version: 11.7
Number of CUDA devices: 2


## Spec

### Data Description

The dataset is a Pandas DataFrame named `data`, which includes the following columns:

- `id`: Unique ID of the variant in the format #chrom:pos:ref:alt (string).
- `#chrom`: Chromosome number where the genetic variant is located.
- `pos`: Position of the genetic variant on the chromosome (integer between 1 and 200,000).
- `ref`: Reference allele (or variant) at the genomic position.
- `alt`: Alternate allele observed at the genomic position.
- `rsids`: Reference SNP cluster ID, a unique identifier for each variant used in the dbSNP database.
- `nearest_genes`: Gene nearest to the variant (string).
- `pval`: P-value, a statistical measure for the strength of evidence against the null hypothesis.
- `mlogp`: Minus log of the p-value, commonly used in genomic studies.
- `beta`: Beta coefficient, representing the effect size of the variant.
- `sebeta`: Standard error of the beta coefficient.
- `af_alt`: Allele frequency of the alternate variant in the general population (float between 0 and 1).
- `af_alt_cases`: Allele frequency of the alternate variant in the cases group (float between 0 and 1).
- `af_alt_controls`: Allele frequency of the alternate variant in the control group (float between 0 and 1).
- `finemapped`: Indicator whether the variant is included in the post-finemapped subset (1) or not (0) (integer).
- `precausal`: Indicator whether the variant is included in the precausal subset (1) or not (0) (integer).
- `causal`: Indicator whether the variant is included in the causal subset (1) or not (0) (integer).
- `trait`: Trait associated with the variant. In this dataset, it refers to the response to the drug paracetamol and NSAIDs.

### Data Subset 
From least likely to be causal (left) to most likely to be causal (right):

GWAS (20 million SNPs) --> Finemapped (3 million SNPs) --> Precausal (1300 SNPs) --> Causal (21 SNPs)

### Task Overview

The task is to build a binary classifier that predicts whether a SNP is included in the causal dataset (i.e., `causal=1`).

### Resources 
- FinnGen Docs: https://finngen.gitbook.io/documentation/
- GWAS YouTube Video: https://www.youtube.com/watch?v=sOP8WacfBM8

In [2]:
dtypes = {
    'id': 'string',
    '#chrom': 'int64',
    'pos': 'int64',
    'ref': 'string',
    'alt': 'string',
    'rsids': 'string',
    'nearest_genes': 'string',
    'pval': 'float64',
    'mlogp': 'float64',
    'beta': 'float64',
    'sebeta': 'float64',
    'af_alt': 'float64',
    'af_alt_cases': 'float64',
    'af_alt_controls': 'float64',
    'finemapped': 'int64',
    'precausal': 'int64',
    'causal': 'int64'
}

data = pd.read_csv('t2d-precausal-2-causal.csv', dtype=dtypes)

# Assert column names
expected_columns = ['#chrom', 'pos', 'ref', 'alt', 'rsids', 'nearest_genes', 'pval', 'mlogp', 'beta',
                    'sebeta', 'af_alt', 'af_alt_cases', 'af_alt_controls', 'finemapped',
                    'id', 'trait', 'causal', 'precausal']
assert set(data.columns) == set(expected_columns), "Unexpected columns in the data DataFrame."

# Assert data types
expected_dtypes = {
    'id': 'string',
    '#chrom': 'int64',
    'pos': 'int64',
    'ref': 'string',
    'alt': 'string',
    'rsids': 'string',
    'nearest_genes': 'string',
    'pval': 'float64',
    'mlogp': 'float64',
    'beta': 'float64',
    'sebeta': 'float64',
    'af_alt': 'float64',
    'af_alt_cases': 'float64',
    'af_alt_controls': 'float64',
    'finemapped': 'int64',
    'precausal': 'int64',
    'causal': 'int64'
}

for col, expected_dtype in expected_dtypes.items():
    assert data[col].dtype == expected_dtype, f"Unexpected data type for column {col}."

In [6]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
df = pd.read_csv('t2d-precausal-2-causal.csv')
scaler = StandardScaler()

In [11]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Assuming you already have a DataFrame 'df' and you want to encode its 'ref' column

# First, we create an instance of the OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)

# Reshape your data as 'fit_transform' expects 2D and 'df['ref']' is 1D
reshaped_data = df['ref'].values.reshape(-1, 1)

# Use the encoder to fit_transform your data
one_hot_encoded_data = one_hot_encoder.fit_transform(reshaped_data)

# The output is an array which you can convert into a DataFrame
df_encoded = pd.DataFrame(one_hot_encoded_data, columns=one_hot_encoder.get_feature_names_out(['ref']))

# Optionally, if you want to add this data back into your original DataFrame, you can do so
df = pd.concat([df, df_encoded], axis=1)


C:\Users\Windows\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [13]:
df.columns

Index(['#chrom', 'pos', 'ref', 'alt', 'rsids', 'nearest_genes', 'pval',
       'mlogp', 'beta', 'sebeta', 'af_alt', 'af_alt_cases', 'af_alt_controls',
       'id', 'finemapped', 'causal', 'precausal', 'trait', 'ref_A',
       'ref_AAAATGATTGG', 'ref_ACAAGTAAATGC', 'ref_AG', 'ref_AT', 'ref_C',
       'ref_CA', 'ref_CAA', 'ref_CT', 'ref_G', 'ref_GA', 'ref_GAA', 'ref_GT',
       'ref_T', 'ref_TA', 'ref_TAA', 'ref_TG', 'ref_TGCA'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
data[['pos', 'mlogp']] = scaler.fit_transform(data[['pos', 'mlogp']])

import seaborn as sns
import matplotlib.pyplot as plt

# Create a color map for finemapped status
colors = {0: 'blue', 1: 'red'}

chromosomes = data['#chrom'].unique()

for chrom in chromosomes:
    data_chrom = data[data['#chrom'] == chrom]
    
    plt.figure(figsize=(20, 12))
    sns.scatterplot(x='pos', y='mlogp', hue='causal', data=data_chrom, palette=colors)
    
    plt.title(f'Chromosome {chrom}: Position vs mlogp (scaled with RobustScaler)')
    plt.xlabel('Position (scaled)')
    plt.ylabel('mlogp (scaled)')
    plt.show()